### Dynamic Pricing & Revenue Optimzation- Cleaning Notebook
##### The objective of this notebook is to clean and validate real-world e-commerce pricing data and analyze the relationship between price, demand, and revenue to support dynamic pricing decisions.

##### Importing Libraries

In [ ]:
import numpy as np #importing numpy for mathematical calculation
import pandas as pd #importing pandas for data manipulation
import matplotlib.pyplot as plt #importing matplotlib for charts and graphs
import seaborn as sns #importing seaborn for charts and graphs


#### Loading the dataset

In [ ]:
df=pd.read_csv("ecommerce_pricing_raw.csv") #loading the dataset
df.head() #displaying the first 5 rows

,product_id,category,base_price,competitor_price,promotion_flag,season,customer_segment,day_of_week,channel,current_price,units_sold
0,1102,Electronics,153.93,23.94,1,Fall,Regular,Tue,Web,111.99,5.0
1,1435,Home,190.17,250.84,0,Spring,Premium,Sat,Mobile App,190.17,179.0
2,1860,Electronics,166.57,257.71,0,Summer,Regular,Mon,Web,166.57,141.0
3,1270,Electronics,215.97,238.38,0,Winter,Regular,Mon,Web,215.97,164.0
4,1106,Fashion,102.55,255.81,1,Spring,Regular,Mon,Web,81.71,1238.0


In [ ]:
df.info() #getting overall information of the dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12240 entries, 0 to 12239
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product_id        12240 non-null  int64  
 1   category          12240 non-null  object 
 2   base_price        12240 non-null  float64
 3   competitor_price  11627 non-null  float64
 4   promotion_flag    12240 non-null  int64  
 5   season            12240 non-null  object 
 6   customer_segment  11872 non-null  object 
 7   day_of_week       12240 non-null  object 
 8   channel           12240 non-null  object 
 9   current_price     12240 non-null  float64
 10  units_sold        12240 non-null  float64
dtypes: float64(4), int64(2), object(5)
memory usage: 1.0+ MB


#####
- Columns such as promotion_flag, and units_sold should be in boolean and integar respectively.
- There are 12240 rows and 11 coulmns total.
- There are some missing values in competitor_price and customer_segment.

In [ ]:
df.describe()

,product_id,base_price,competitor_price,promotion_flag,current_price,units_sold
count,12240.000000,12240.000000,11627.000000,12240.000000,12240.000000,12240.000000
mean,1502.492810,154.061050,154.101742,0.252778,146.483073,682.826634
std,289.848111,84.579864,83.899477,0.434622,82.057021,4652.450774
min,1000.000000,0.000000,10.030000,0.000000,-1.000000,0.000000
25%,1249.000000,81.730000,82.240000,0.000000,76.397500,52.000000
50%,1505.000000,154.335000,153.790000,0.000000,144.590000,123.000000
75%,1755.250000,227.252500,227.290000,1.000000,213.675000,291.250000
max,1999.000000,299.990000,299.970000,1.000000,299.990000,260410.000000


In [ ]:
df.duplicated().sum()

np.int64(240)

#### There are total 240 duplicated rows that need to be removed.

In [ ]:
df.isna().sum()

,0
product_id,0
category,0
base_price,0
competitor_price,613
promotion_flag,0
season,0
customer_segment,368
day_of_week,0
channel,0
current_price,0


####
- There are total 613 values missing in competitor_price which should be filled using median after grouping by category.
- There are total 368 values missing in customer_segment, which should be filled by mode after grouping by category.

##### Data Assessment Summary
- Missing competitor prices

- Missing customer segments

- Duplicate records

- Invalid prices

- Extreme demand outliers

##### Converting Datatypes

In [ ]:
df['promotion_flag']= df['promotion_flag'].astype(bool) #Converting datatype in boolean
df['units_sold']= df['units_sold'].round().astype(int) #Converting datatype in integar

- Units sold are whole numbers

- Floats appear due to aggregation / noise

##### Removing duplicates

##### Duplicate transactional records were removed to prevent double counting of demand and revenue.

In [ ]:
df.drop_duplicates(inplace=True)

#### Removing invalid prices

In [ ]:
df= df[(df['base_price']>0) & (df['current_price']>0)]

##### Fixing missing competitor prices (category-wise median)

In [ ]:
df['competitor_price']= df.groupby('category')['competitor_price'].transform(lambda x: x.fillna(x.median()))

##### Fixing missing customer segments (category-wise mode)

In [ ]:
df['customer_segment']= df.groupby('category')['customer_segment'].transform(lambda x: x.fillna(x.mode()[0]))

#### Cap extreme demand values

In [ ]:
upper_cap= df['units_sold'].quantile(0.99)
df['units_sold']= np.where(df['units_sold']>upper_cap, upper_cap, df['units_sold'])

#### Final Check

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11761 entries, 0 to 11999
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   product_id        11761 non-null  int64  
 1   category          11761 non-null  object 
 2   base_price        11761 non-null  float64
 3   competitor_price  11761 non-null  float64
 4   promotion_flag    11761 non-null  bool   
 5   season            11761 non-null  object 
 6   customer_segment  11761 non-null  object 
 7   day_of_week       11761 non-null  object 
 8   channel           11761 non-null  object 
 9   current_price     11761 non-null  float64
 10  units_sold        11761 non-null  float64
dtypes: bool(1), float64(4), int64(1), object(5)
memory usage: 1022.2+ KB


In [ ]:
df.describe()

,product_id,base_price,competitor_price,current_price,units_sold
count,11761.000000,11761.000000,11761.000000,11761.000000,11761.000000
mean,1503.023212,155.862281,153.948519,148.048798,480.221070
std,289.840873,83.680391,81.754158,81.292939,1334.829847
min,1000.000000,10.000000,10.030000,7.720000,0.000000
25%,1250.000000,84.190000,85.810000,78.330000,52.000000
50%,1507.000000,156.080000,154.200000,146.260000,123.000000
75%,1756.000000,228.370000,222.330000,214.720000,291.000000
max,1999.000000,299.990000,299.970000,299.990000,9960.000000


In [ ]:
df.to_csv("ecommerce_pricing_cleaned.csv", index=False)

##### The dataset is now free of duplicates, missing values, invalid prices, and extreme noise, making it suitable for pricing and revenue optimization analysis.